In [1]:
from pipeline import TrainingPipeline
from data_preprocessing.feature_engineering import FeatureEngineer
from data_preprocessing.outliers_detection import OutlierProcessing
from training.data_splitting import DataSplitter

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
cleaned_data = TrainingPipeline().training_pipeline()
cleaned_data

c:\projects\ml_learn\End-To-End-Projects\Electricity_Consumption_Predictor\extract_ingest\ingest.py:25: DtypeWarning: Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filePath, delimiter=';')
c:\projects\ml_learn\End-To-End-Projects\Electricity_Consumption_Predictor\data_preprocessing\type_convert.py:8: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], errors="coerce")
c:\projects\ml_learn\End-To-End-Projects\Electricity_Consumption_Predictor\data_preprocessing\initial_process.py:20: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['Datetime'] = df['Datetime'].dt.floor('H')


,Datetime,Global_active_power,Global_reactive_power,Sub_metering_1,Sub_metering_2,Sub_metering_3,Voltage
0,2006-12-16 17:00:00,152.024,8.244,0.0,19.0,607.0,234.643889
1,2006-12-16 18:00:00,217.932,4.802,0.0,403.0,1012.0,234.580167
2,2006-12-16 19:00:00,204.014,5.114,0.0,86.0,1001.0,233.232500
3,2006-12-16 20:00:00,196.114,4.506,0.0,0.0,1007.0,234.071500
4,2006-12-16 21:00:00,183.388,4.600,0.0,25.0,1033.0,237.158667
...,...,...,...,...,...,...,...
34584,2010-11-26 17:00:00,103.554,3.684,0.0,0.0,772.0,237.069667
34585,2010-11-26 18:00:00,94.408,3.222,0.0,0.0,0.0,237.531833
34586,2010-11-26 19:00:00,99.560,3.602,0.0,4.0,0.0,236.741000
34587,2010-11-26 20:00:00,69.822,3.670,0.0,64.0,0.0,239.396000


In [3]:
cleaned_data['hour'] = cleaned_data['Datetime'].dt.hour
cleaned_data['day_of_week'] = cleaned_data['Datetime'].dt.day_of_week
cleaned_data['month'] = cleaned_data['Datetime'].dt.month
cleaned_data['is_weekend'] = cleaned_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Summer"
    elif month in [6, 7, 8, 9]:
        return "Monsoon"
    else:
        return "Autumn"

# Apply function to create season column
cleaned_data['season'] = cleaned_data['month'].apply(get_season)
cleaned_data.drop(columns=['Datetime', "Sub_metering_1", "Sub_metering_2", "Sub_metering_3", "Global_reactive_power"], inplace=True)

In [4]:
cleaned_data

,Global_active_power,Voltage,hour,day_of_week,month,is_weekend,season
0,152.024,234.643889,17,5,12,1,Winter
1,217.932,234.580167,18,5,12,1,Winter
2,204.014,233.232500,19,5,12,1,Winter
3,196.114,234.071500,20,5,12,1,Winter
4,183.388,237.158667,21,5,12,1,Winter
...,...,...,...,...,...,...,...
34584,103.554,237.069667,17,4,11,0,Autumn
34585,94.408,237.531833,18,4,11,0,Autumn
34586,99.560,236.741000,19,4,11,0,Autumn
34587,69.822,239.396000,20,4,11,0,Autumn


In [5]:
f1 = FeatureEngineer()
featured_data = f1.execute_transformation(cleaned_data, 'Voltage', "StandardScalling")
featured_data = f1.execute_transformation(featured_data, 'Global_active_power', "StandardScalling")
featured_data = f1.execute_transformation(featured_data, 'season', "LabelEncoding")

In [6]:
cleaned_data = OutlierProcessing().process_outliers(featured_data, featured_data.columns, "IQROutlierDetection")
cleaned_data

,Global_active_power,Voltage,hour,day_of_week,month,is_weekend,season
0,152.024,-2.077126,17,5,12,1,3
3,196.114,-2.269027,20,5,12,1,3
4,183.388,-1.234013,21,5,12,1,3
5,132.008,-0.697144,22,5,12,1,3
6,123.696,-0.073666,23,5,12,1,3
...,...,...,...,...,...,...,...
34584,103.554,-1.263851,17,4,11,0,0
34585,94.408,-1.108904,18,4,11,0,0
34586,99.560,-1.374041,19,4,11,0,0
34587,69.822,-0.483917,20,4,11,0,0


In [7]:
X, y, X_train, X_test, y_train, y_test = DataSplitter().split(cleaned_data, "Global_active_power", "simple")
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(26234, 6) (6559, 6) (26234,) (6559,)


In [28]:
model = LinearRegression()
model.fit(X_train, y_train)

y_hat_train = model.predict(X_train).reshape(-1, 1)
y_hat_test = model.predict(X_test).reshape(-1, 1)

print("Train Error\n", mean_squared_error(y_train, y_hat_train))
print("Test Error\n", mean_squared_error(y_test, y_hat_test))

X_min = X_train.min()
X_max = X_train.max()

if len(X_train.shape) == 1 or X_train.shape[1] == 1:
    X_lin_1d = np.linspace(X_min, X_max, 100).reshape(-1, 1)
    y_hat_lin = model.predict(X_lin_1d)

    plt.plot(X_train, y_train, 'o', label='Train', markersize=1)
    plt.plot(X_test, y_test, 'o', label='Test', ms=3)
    plt.plot(X_lin_1d, y_hat_lin, label='Model', color='black')
    plt.legend()
    plt.xlabel("X")
    plt.ylabel("y")
    plt.title("Linear Regression Fit")
    plt.show()
else:
    print("Cannot plot: Model has more than one feature.")


ValueError: could not convert string to float: 'Summer'